In [1]:
!wget https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/title_conference.csv

--2021-12-12 16:40:53--  https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/title_conference.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 188402 (184K) [text/plain]
Saving to: ‘title_conference.csv’

title_conference.cs 100%[===================>] 183.99K  --.-KB/s    in 0.02s   

2021-12-12 16:40:54 (11.8 MB/s) - ‘title_conference.csv’ saved [188402/188402]



In [3]:
# !pip install transformers

     |████████████████████████████████| 3.3 MB 8.8 MB/s 
     |████████████████████████████████| 61 kB 452 kB/s 
     |████████████████████████████████| 596 kB 50.8 MB/s 
     |████████████████████████████████| 3.3 MB 46.3 MB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Exploring & preprocessing data

In [7]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd

In [8]:
df = pd.read_csv('title_conference.csv')
df.head()

,Title,Conference
0,Innovation in Database Management: Computer Sc...,VLDB
1,High performance prime field multiplication fo...,ISCAS
2,enchanted scissors: a scissor interface for su...,SIGGRAPH
3,Detection of channel degradation attack by Int...,INFOCOM
4,Pinning a Complex Network through the Betweenn...,ISCAS


In [10]:
df['Conference'].value_counts()
#we see the classes are imbalanced

ISCAS       864
INFOCOM     515
VLDB        423
WWW         379
SIGGRAPH    326
Name: Conference, dtype: int64

In [12]:
possible_labels = df.Conference.unique()
possible_labels

array(['VLDB', 'ISCAS', 'SIGGRAPH', 'INFOCOM', 'WWW'], dtype=object)

In [13]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'INFOCOM': 3, 'ISCAS': 1, 'SIGGRAPH': 2, 'VLDB': 0, 'WWW': 4}

In [14]:
df['label'] = df.Conference.replace(label_dict)

In [15]:
df.head()

,Title,Conference,label
0,Innovation in Database Management: Computer Sc...,VLDB,0
1,High performance prime field multiplication fo...,ISCAS,1
2,enchanted scissors: a scissor interface for su...,SIGGRAPH,2
3,Detection of channel degradation attack by Int...,INFOCOM,3
4,Pinning a Complex Network through the Betweenn...,ISCAS,1


### Training and Validation split
As the data is unbalanced we perform a stratifies split

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [17]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [18]:
df.groupby(['Conference', 'label', 'data_type']).count()

Title
Conference label data_type       
INFOCOM    3     train        438
                 val           77
ISCAS      1     train        734
                 val          130
SIGGRAPH   2     train        277
                 val           49
VLDB       0     train        359
                 val           64
WWW        4     train        322
                 val           57

### Bert tokenization and encoding the data

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]